In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils import clip_grad_norm_
import numpy as np
import pandas as pd
import jieba
from tqdm import tqdm
from torchtext import data, datasets
from torchsummary import summary
from gensim.models import KeyedVectors

jieba.load_userdict('../word2vec/500000-dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.510 seconds.
Prefix dict has been built succesfully.


In [2]:
'''
词典类，记录数据集中数显的词汇及其索引和数量
'''
class Dictionary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.word2count = {}
        
    def add_word(self, word):
        # build word - count
        if word not in self.word2count:
            self.word2count[word] = 1
        else:
            self.word2count[word] += 1
    
    '''
    仅使用训练集中出现次数 >= 阈值的词汇构建词表
    '''
    def cut_dict(self, cut=2):
        self.word2idx = {'<PAD>':0, '<UNK>':1}
        self.idx2word = {0:'<PAD>', 1:'<UNK>'}
        word2count = {}
        idx = 2
        
        for word in self.word2count.keys():
            if self.word2count[word] >= cut:
                self.word2idx[word] = idx
                self.idx2word[idx] = word
                idx += 1
                word2count[word] = self.word2count[word]
                
        self.word2count = word2count
        

'''
构建停止词表
'''
stop_words = []
with open('../word2vec/stopwords.txt') as f:
    for line in f:
        stop_words.append(line.strip())

In [3]:
'''
对评论进行中文分词
'''
def seg(data_frame):
    segs = jieba.cut(data_frame['review'], cut_all=False)
    l = []
    for seg in segs:
        if seg not in stop_words and seg != ' ':
            l.append(seg)
    return l
    
df = pd.read_csv('../datasets/online_shopping_10_cats.csv')
df = df[['label', 'review']].dropna().copy()  # 删除具有缺省的行

df['review'] = df.apply(seg, axis=1)

'''
简单的样本数据统计
'''
length_list = [len(review) for review in df['review']]
print('数据集样本最小长度: {}\n'
      '数据集样本最大长度: {}\n'
      '数据集样本平均长度: {:.1f}'
      .format(min(length_list), max(length_list), sum(length_list)/len(length_list)))

print('正样本数量: {}\n'
      '负样本数量: {}'
      .format(len(df[df['label']==1]), len(df[df['label']==0])))

数据集样本最小长度: 1
数据集样本最大长度: 1275
数据集样本平均长度: 26.0
正样本数量: 31727
负样本数量: 31046


In [4]:
'''
加载词向量
'''
vector = KeyedVectors.load_word2vec_format('../word2vec/500000-small.txt')

/home/fivesheep/anaconda3/envs/Pytorch11-Py36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: [Errno 2] No such file or directory: '../word2vec/500000-small.txt'

In [ ]:
'''
获取数据集字典
'''

dictionary = Dictionary()

for review in df['review']:
    for i, word in enumerate(review):
        dictionary.add_word(word)
        
# 使用出现次数至少为 2 的词构建词典
dictionary.cut_dict(2)

In [ ]:
'''
对训数据中的词汇进行统计
'''
em_word = 0
for word in dictionary.word2idx.keys():
    if word in vector:
        em_word += 1

print('词汇总量为: {}'
      '\n具有词向量的词汇量为: {}'
      '\n占比为: {:.2f}%\n'
      .format(len(dictionary.word2idx), em_word, em_word/len(dictionary.word2idx)*100))

'''
对训练集中的词汇数进行统计
'''
# 按照词汇出现数量进行排序，大→小
dict_list = [(key, value) for key, value in dictionary.word2count.items()]
dict_list.sort(key=lambda x:x[1], reverse=True)

# 统计词数总量和具有词向量的词汇量
total_count = 0
em_count = 0
for d in dict_list:
    total_count += d[1]
    if d[0] in vector:
        em_count += d[1]
print('词汇总数为: {}'
      '\n具有词向量的词汇量为: {}'
      '\n占比为: {:.2f}%'
      .format(total_count, em_count, em_count/total_count*100))

In [ ]:
'''
构建训练集、测试集序列化数据
'''
EMBEDDING_DIM = 200
# 词嵌入矩阵
embedding_matrix = np.zeros((len(dictionary.word2idx), EMBEDDING_DIM), dtype=np.float32)

# 依据 word2idx 的映射，为 embedding_matrix 填充词向量
texts = []
labels = []
for review in df['review']:
    text = []
    for i, word in enumerate(review):
        # 如果在字典中没有找到该词汇，设置为 1 即 <UNK>
        embedding_index = dictionary.word2idx.get(word, 1)
        text.append(embedding_index)
        # 如果该词有预训练词向量，初始化
        if word in vector:
            embedding_matrix[embedding_index, :] = vector[word]
    texts.append(text)

# 构建词嵌入 Tensor
embedding_matrix = torch.Tensor(embedding_matrix)


for label in df['label']:
    labels.append(label)
    
'''
对数据集进行 padding/truncation 操作, pre / post
'''
FIXED_LENGTH = 25

def pad_truncate(dataset, length=50, type='pre'):
    fixed_texts = []
    
    for text in dataset:
        fixed_text = []
        
        if len(text) < length:
            fixed_text = text[:] + [0] * (length-len(text))
        else:
            if type == 'pre':  # 截取前面 length 个词
                fixed_text = text[:length]
            elif type == 'post':  # 截取后面 length 个词
                fixed_text = text[-length:]
                
        fixed_texts.append(fixed_text)
        
    return fixed_texts

texts = pad_truncate(texts, length=FIXED_LENGTH, type='pre')

In [ ]:
'''
划分训练集、验证集(5000)
'''
np.random.seed(1)
data_size = len(texts)
indices = np.arange(data_size)
np.random.shuffle(indices)

# 训练集
train_texts = np.array(texts)[indices[:-5000]]
train_labels = np.array(labels)[indices[:-5000]]
# 验证集
val_texts = np.array(texts)[indices[-5000:]]
val_labels = np.array(labels)[indices[-5000:]]

train_x = torch.Tensor(train_texts).long()
train_y = torch.Tensor(train_labels).long()
val_x = torch.Tensor(val_texts).long()
val_y = torch.Tensor(val_labels).long()

In [215]:
'''
TextCNN 模型
'''
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def create_embedding_layer(embedding_matrix, trainable=True):
        vocab_size, embed_dim = embedding_matrix.size()
        embed_layer = nn.Embedding(vocab_size, embed_dim)
        embed_layer.load_state_dict({'weight': embedding_matrix})
        embed_layer.weight.requires_grad = trainable
        return embed_layer, vocab_size, embed_dim
    
class TextCNN(nn.Module):
    def __init__(self, embedding_matrix):
        super(TextCNN, self).__init__()
        self.embed, self.vocab_size, self.embed_dim = create_embedding_layer(embedding_matrix, trainable=True)
        
        self.conv1d_1 = nn.Conv1d(200, 100, 2) # 输入的 channels 为 embedding_dim
        self.conv1d_2 = nn.Conv1d(200, 100, 3)
        self.conv1d_3 = nn.Conv1d(200, 100, 4)
        
        self.relu = nn.ReLU()
        
        self.maxpool_1 = nn.AdaptiveMaxPool1d(1)
        self.maxpool_2 = nn.AdaptiveMaxPool1d(1)
        self.maxpool_3 = nn.AdaptiveMaxPool1d(1)
        
        self.dropout_1 = nn.Dropout(0.7)
        self.fc_1 = nn.Linear(100*3, 32)
        self.dropout_2 = nn.Dropout(0.7)
        self.fc_2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.embed(x)
        # (bs, sentence_len, embedding_dim) → (bs, embedding_dim, sentence_len)
        out = out.permute([0, 2, 1])
        
        out_1 = self.conv1d_1(out)
        out_2 = self.conv1d_2(out)
        out_3 = self.conv1d_3(out)
        # (bs, 64, sentence_len - filter_size + 1)
        
        out_1 = self.relu(out_1)
        out_2 = self.relu(out_2)
        out_3 = self.relu(out_3)
        
        # 在第 3 维度上取最大值 (bs, 64, 1)
        out_1 = self.maxpool_1(out_1).squeeze(2)
        out_2 = self.maxpool_2(out_2).squeeze(2)
        out_3 = self.maxpool_3(out_3).squeeze(2)
        
        cat = torch.cat((out_1, out_2, out_3), dim=1)
        
        out = self.dropout_1(cat)
        out = self.fc_1(out)
        out = self.dropout_2(out)
        out = self.fc_2(out)
        return out

In [219]:
'''
手动进行 SGD 训练
'''
# hyperparameters
batch_size = 32
epochs = 10
learning_rate = 0.0003

'''
使用 TensorDataset 和 DataLoader 存放数据集
'''
# 加载为 TensorDataset 对象
train_ds = TensorDataset(train_x, train_y)
val_ds = TensorDataset(val_x, val_y)

# 加载为 DataLoader 对象
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=len(val_ds), shuffle=False)

# initialize Model
model = TextCNN(embedding_matrix).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# fit
for epoch in range(epochs):
    model.train()
    
    with tqdm(enumerate(train_dl)) as t:
        for i, (x, y) in t:
            x = x.to(device)
            y = y.to(device)
            
            out = model(x)
            loss = criterion(out, y)

            # backprop and optimizer
            model.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(), 2.0)
            optimizer.step()

            if (i+1) % 50 == 0:
                print('Epoch: [{}/{}], 正在训练：[{}/{}] 个样本'
                      .format(epoch+1, epochs, (i+1)*batch_size, len(train_x)))

                with torch.no_grad():
                    out_ = model(x)
                    correct = torch.sum(torch.argmax(out_, dim=1) == y).item()
                    print('Val: {:.3f}%'.format(correct/batch_size*100))
t.close()

72it [00:00, 111.44it/s]

Epoch: [1/10], 正在训练：[1600/57773] 个样本
Val: 71.875%


117it [00:01, 109.91it/s]

Epoch: [1/10], 正在训练：[3200/57773] 个样本
Val: 62.500%


161it [00:01, 100.02it/s]

...

Epoch: [10/10], 正在训练：[54400/57773] 个样本
Val: 96.875%


1771it [00:16, 109.17it/s]

Epoch: [10/10], 正在训练：[56000/57773] 个样本
Val: 93.750%


1806it [00:16, 106.48it/s]

Epoch: [10/10], 正在训练：[57600/57773] 个样本
Val: 90.625%


In [231]:
'''
训练集 Accuracy
'''
with torch.no_grad():
    correct = 0
    
    for _, (x_, y_) in enumerate(train_dl):
        out_ = model(x_.to(device))
        correct += torch.sum(torch.argmax(out_, dim=1) == y_.to(device)).item()
        
    print('训练集 Acc: {:.3f}%'.format(correct / len(train_x) * 100))

'''
验证集 Accuracy
'''
with torch.no_grad():
    correct = 0
    
    for _, (x_, y_) in enumerate(val_dl):
        out_ = model(x_.to(device))
        correct += torch.sum(torch.argmax(out_, dim=1) == y_.to(device)).item()
        
    print('验证集 Acc: {:.3f}%'.format(correct / len(val_x) * 100))

训练集 Acc: 98.859%
验证集 Acc: 89.460%
